In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from statsmodels.tsa.stattools import acf,pacf
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

KeyboardInterrupt: 

In [2]:
south_east = pd.read_csv("Brazil Data/southeast.csv")

In [3]:
max_date = south_east['Data'].max()
min_date = south_east['Data'].min()

stations = south_east['station_code'].unique()

print(min_date, max_date)
print(stations)

start_date = '2020-04-30'

2000-05-07 2021-04-30
['A047' 'A757' 'A929' 'A002' 'A926' 'A754' 'A752' 'A919' 'A031' 'A717'
 'A916' 'A025' 'A910' 'A936' 'A720' 'A732' 'A703' 'A731' 'A024' 'A927'
 'A016' 'A930' 'A921' 'A931' 'A915' 'A904' 'A721' 'A906' 'A756' 'A749'
 'A704' 'A032' 'A902' 'A023' 'A923' 'A017' 'A751' 'A759' 'A903' 'A935'
 'A907' 'A028' 'A941' 'A750' 'A911' 'A042' 'A933' 'A901' 'A934' 'A723'
 'A014' 'A924' 'A012' 'A034' 'A026' 'A730' 'A022' 'A743' 'A909' 'A046'
 'A761' 'A045' 'S703' 'A742' 'A928' 'S717' 'A056' 'A914' 'A722' 'A905'
 'A015' 'A003' 'A937' 'S702' 'S705' 'A037' 'A035' 'A702' 'A719' 'S712'
 'A760' 'A036' 'A917' 'A758' 'A922' 'S707' 'A912' 'A001' 'A908' 'A724'
 'A913' 'A710' 'S710' 'A027' 'S701' 'S706' 'A013' 'A011' 'A033' 'A932'
 'A005' 'S709' 'S714' 'S708' 'S715' 'A029' 'S711' 'S713' 'S716' 'A920'
 'S704' 'A709' 'A944' 'A942' 'A943']


In [4]:
renamed_columns_en = ['date','hour','total precipitation (mm)','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)','max. temperature in the previous hour (°c)','min. temperature in the previous hour (°c)','dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','relative humidity max. in the previous hour (%)','relative humidity min. in the previous hour (%)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)','wind speed (m/s)','region','state','station','station_code','latitude','longitude','height']

def process_raw(df):
    """
    ! drop index
    ! rename the columns
    ! combine hour and date columns -> transform to date_time
    ! append columns data from all stations to keep the data granularity
    ! remove not useful columns
    ! return table 
    """
    df.drop(['index'],inplace=True, axis=1)
    df.columns = renamed_columns_en
    df['date_time'] = pd.to_datetime(df['date'] + ' ' +  df['hour'])
    df.drop(['date','hour','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)', 'dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)'], inplace=True, axis=1)

    # station_codes = list(df.station_code.unique())
    # df_models = by_code(df,station_codes[0])
    # if len(station_codes)>1:
    #     for i in station_codes[1:]:
    #         df_temp = by_code(df,i)
    #         df_models = pd.concat([df_models,df_temp],axis=1)
    return df

# def by_code(df, station_code):
#     """
#     ! function that filters the dataframe by station_code and rename the columns by the status code
#     """
#     df_station = None
#     df_station = df[df['station_code'] == station_code]
#     df_station.index = df_station.iloc[:,-1]
#     df_station.drop(['station_code','date_time'],axis=1,inplace=True)
#     df_station.columns = list(map(lambda x: station_code+'_'+x, df_station.columns))
    return df_station

def clean_na(df, na_value = -9999):
    """ 
    ! replace Na from dataset
    """
    df = df.replace(to_replace=na_value,value=np.NaN)
    df = df[df.iloc[:,1].first_valid_index():]
    df = df.fillna(method='ffill')
    return df

def make_dataset(stations, start_date, df):
    """
    ! filter raw data from date and stations code
    ! process data
    ! clean na
    """
    df = df[df['Data'] >= start_date]
    df = df[df['station_code'].apply(lambda x: x in stations)]
    df = process_raw(df)
    df = clean_na(df)
    return df.reset_index()   


In [11]:
south_east_Clean = make_dataset(stations,'2019-04-03', south_east)
south_east_Clean

# the start_date variable I set up produces an empty table

,index,total precipitation (mm),max. temperature in the previous hour (°c),min. temperature in the previous hour (°c),relative humidity max. in the previous hour (%),relative humidity min. in the previous hour (%),wind speed (m/s),region,state,station,station_code,latitude,longitude,height,date_time
0,3128690,0.0,26.7,25.6,29.0,27.0,3.0,CO,MS,RIBAS DO RIO PARDO,S715,-20.466694,-53.763028,442.00,2020-05-28 18:00:00
1,3128691,0.0,26.7,26.1,32.0,28.0,2.8,CO,MS,RIBAS DO RIO PARDO,S715,-20.466694,-53.763028,442.00,2020-05-28 19:00:00
2,3128692,0.0,26.4,25.3,37.0,30.0,1.9,CO,MS,RIBAS DO RIO PARDO,S715,-20.466694,-53.763028,442.00,2020-05-28 20:00:00
3,3128693,0.0,25.3,21.4,47.0,36.0,0.9,CO,MS,RIBAS DO RIO PARDO,S715,-20.466694,-53.763028,442.00,2020-05-28 21:00:00
4,3128694,0.0,21.9,20.8,49.0,45.0,1.3,CO,MS,RIBAS DO RIO PARDO,S715,-20.466694,-53.763028,442.00,2020-05-28 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638802,3767492,0.0,22.0,21.1,74.0,70.0,0.8,CO,MS,BATAGUASSU,A759,-21.750123,-52.471294,392.13,2021-04-30 19:00:00
638803,3767493,0.0,22.0,21.1,74.0,70.0,0.8,CO,MS,BATAGUASSU,A759,-21.750123,-52.471294,392.13,2021-04-30 20:00:00
638804,3767494,0.0,22.0,21.1,74.0,70.0,0.8,CO,MS,BATAGUASSU,A759,-21.750123,-52.471294,392.13,2021-04-30 21:00:00
638805,3767495,0.0,22.0,21.1,74.0,70.0,0.8,CO,MS,BATAGUASSU,A759,-21.750123,-52.471294,392.13,2021-04-30 22:00:00


In [35]:
south_east_Clean.groupby(by=[south_east_Clean['date_time'].dt.year, 
    south_east_Clean['date_time'].dt.month,
    south_east_Clean['date_time'].dt.day ]).mean()


index  total precipitation (mm)  \
date_time date_time date_time                                           
2020      1         1          3.286862e+06                  0.103472   
                    2          3.286887e+06                  0.149537   
                    3          3.286911e+06                  0.378241   
                    4          3.286935e+06                  0.626157   
                    5          3.286959e+06                  0.415509   
...                                     ...                       ...   
2021      4         26         3.606108e+06                  0.012684   
                    27         3.606132e+06                  0.029794   
                    28         3.606156e+06                  0.045796   
                    29         3.606180e+06                  0.047271   
                    30         3.606204e+06                  0.038938   

                               max. temperature in the previous hour (°c)  \
date_time date_time date_time                                               
2020      1         1                                           26.500116   
                    2                                           26.281597   
                    3                                           25.908449   
                    4                                           25.190741   
                    5                                           25.453704   
...                                                                   ...   
2021      4         26                                          25.104830   
                    27                                          24.901217   
                    28                                          24.796940   
                    29                                          24.615044   
                    30                                          24.676106   

                               min. temperature in the previous hour (°c)  \
date_time date_time date_time                                               
2020      1         1                                           25.154514   
                    2                                           25.025000   
                    3                                           24.575810   
                    4                                           24.088773   
                    5                                           24.282870   
...                                                                   ...   
2021      4         26                                          23.673820   
                    27                                          23.409218   
                    28                                          23.249779   
                    29                                          23.121460   
                    30                                          23.223968   

                               relative humidity max. in the previous hour (%)  \
date_time date_time date_time                                                    
2020      1         1                                                75.299769   
                    2                                                76.321759   
                    3                                                78.409722   
                    4                                                79.521991   
                    5                                                78.189815   
...                                                                        ...   
2021      4         26                                               69.712389   
                    27                                               68.281342   
                    28                                               66.539454   
                    29                                               65.171829   
                    30                                               66.672935   

                 